# Regresión Lineal Múltiple

Ya teniendo clara la regresión lineal simple **sin importar si el método es por descenso del gradiente o por mínimos cuadrados con resolución de ecuaciones lineales con dos incógnitas**, es fácil comprender la múltiple. En la simple hay $m$ ejemplos de una sola característica de entrada, en la múltiple hay $m$ conjuntos de ejemplos de dos o más características. Cada característica tendrá su respectiva derivada parcial.

La regresión lineal múltiple se entiende de la siguiente forma:
$$\begin{equation}
f_{\vec{w},b}(\vec{x}) = w_{1}*x_{1} + w_{2}*x_{2} + ... + w_{n}*x_{n} + b
\end{equation}
$$
teniendo en cuenta que $\vec{w}$ es un vector de pesos, cuya longitud es igual a la de cualquier vector de ejemplo. El vector de pesos sería: $\vec{w} = [w_{1}, w_{2}, ..., w_{n}]$ y sobre un ejemplo x es un vector: $\vec{x} = [x_{1}, x_{2}, ..., x_{n}]$
que pertenece a una matriz X:
$X_{m,n} = \begin{equation}
\begin{bmatrix}
x_{11} & x_{12} & ... & x_{1n}\\
x_{21} & x_{22} & ... & x_{2n}\\
. & . & . & .\\
. & . & x_{ij} & .\\
. & . & . & .\\
x_{m1} & x_{m2} & ... & x_{mn}
\end{bmatrix}
\end{equation}$

La matriz de ejemplos tendrá una dimensión de $(m,n)$, que al ser hecho el producto punto por el vector $\vec{w}$ este vector debe ser de dimensión $(n,1)$ y el resultado, que será de $(m,1)$, se les debe sumar el escalar $b$.

De manera que la derivada respecto a $b$ sigue siendo como la de una regresión simple, pero la derivada respecto a w sería:

$$
\begin{equation}
\frac{\partial{J_{\vec{w}, b}}(\vec{w}, b)}{\partial{w}} = -\frac{2}{m}\sum_{i=1}^{m}(y - wx -b)*x_{j}^{i}
\end{equation}
$$
donde i es el vector ejemplo i-ésimo y j es el dato en la posición j-ésima del vector $x^{i}$, es decir, $x_{j}^{i}$ representa un escalar, y tiene todo el sentido porque vamos a intentar calcular la derivada.

La derivada respecto al sesgo $b$ es:
$$
\begin{equation}
\frac{\partial{J_{\vec{w}, b}}(\vec{w}, b)}{\partial{b}} = -\frac{2}{m}\sum_{i=1}^{m}(y - wx -b)
\end{equation}
$$

In [1]:
import ipynb
import numpy as np
import matplotlib.pyplot as plt
import copy, math

Usaremos numpy para mejorar los tiempos de cálculo de matrices.

In [2]:
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

b_init = 785.1811367994083
w_init = np.array([ 0.39133535, 18.75376741, -53.36032453, -26.42131618])

In [3]:
def predecir(x, w, b):
    prediccion = np.dot(x, w) + b
    return prediccion

def costo(X, y, w, b): 
    m = X.shape[0]
    costo = 0.0
    for i in range(m):                                
        y_hat = np.dot(X[i], w) + b           
        costo = costo + (y[i] - y_hat)**2      
    costo = costo / (2*m)                         
    return costo

cost = costo(X_train, y_train, w_init, b_init)
print(f'Costo a este w : {cost}')

Costo a este w : 1.5578904045996674e-12


In [4]:
def calcular_gradiente(x, y, w, b):
    m, n = x.shape
    dj_dw = np.zeros((n,))
    dj_db = 0.

    for i in range(m):                             
        err = (np.dot(x[i], w) + b) - y[i]   
        for j in range(n):                         
            dj_dw[j] = dj_dw[j] + err * x[i, j]    
        dj_db = dj_db + err                        
    dj_dw = dj_dw / m                                
    dj_db = dj_db / m                                
        
    return dj_db, dj_dw

In [5]:
def regresion_multiple(x, y, w_in, b_in, costo, gradiente, alpha, epocas):
    J_history = []
    w = copy.deepcopy(w_in) 
    b = b_in
    for i in range(epocas):
        dj_db, dj_dw = gradiente(x, y, w, b)
        # actualizamos pesos y el sesgo
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        if i < 100000:
            J_history.append(costo(x, y, w, b))
        if i % math.ceil(epocas/10) == 0:
            print(f"Iteración {i:4d}: Costo {J_history[-1]:8.2f}")
    return w, b, J_history


In [6]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.
# some gradient descent settings
iterations = 1000
alpha = 5.0e-7
# run gradient descent 
w_final, b_final, J_hist = regresion_multiple(X_train, y_train, initial_w, initial_b, costo, calcular_gradiente, alpha, iterations)
print(f"b,w found by gradient descent: {b_final:0.2f},{w_final} ")
m,_ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

Iteración    0: Costo  2529.46
Iteración  100: Costo   695.99
Iteración  200: Costo   694.92
Iteración  300: Costo   693.86
Iteración  400: Costo   692.81
Iteración  500: Costo   691.77
Iteración  600: Costo   690.73
Iteración  700: Costo   689.71
Iteración  800: Costo   688.70
Iteración  900: Costo   687.69
b,w found by gradient descent: -0.00,[ 0.20396569  0.00374919 -0.0112487  -0.0658614 ] 
prediction: 426.19, target value: 460
prediction: 286.17, target value: 232
prediction: 171.47, target value: 178


Ahora miremos la otra forma, la matricial para resolver para muchas incógnitas.

## Forma matricial

Si tenemos la forma de nuestra función lineal múltiple y la volvemos en forma matricial: $y = X_{(m,n)}w_{(n,1)} + e_{(m,1)}$.

Recordemos que: $X_{m,n} = \begin{equation}
\begin{bmatrix}
x_{11} & x_{12} & ... & x_{1n}\\
x_{21} & x_{22} & ... & x_{2n}\\
. & . & . & .\\
. & . & x_{ij} & .\\
. & . & . & .\\
x_{m1} & x_{m2} & ... & x_{mn}
\end{bmatrix}
\end{equation}$
pero como hay que colocar el $w_0$(este es el sesgo o intercepto $b$ que hemos utilizado anteriormente) en el vector $w$ y sería de dimensión $(n+1,1)$, colocamos una columna de 1's a la izquierda, así:
$X_{m,n+1} = \begin{equation}
\begin{bmatrix}
1 & x_{11} & x_{12} & ... & x_{1n}\\
1 & x_{21} & x_{22} & ... & x_{2n}\\
. & . & . & . & .\\
. & . & . & x_{ij} & .\\
. & . & . & . & .\\
1 & x_{m1} & x_{m2} & ... & x_{mn}
\end{bmatrix}
\end{equation}$
por lo que se convertiría en una matriz $(m,n+1)$

se puede demostrar que minimizando la suma de los errores se puede proveer las ecuaciones normales: $X^{T}Xw = X^{T}y$,

donde $\begin{equation}w = \begin{bmatrix} w_0\\ w_{1}\\ .\\ .\\ .\\ w_{m} \end{bmatrix} = (X^{T}X)^{-1}X^{T}y \end{equation}$

$\begin{equation}
\hat{y} = \begin{bmatrix}
\hat{y_{1}}\\
\hat{y_{2}}\\
.\\
.\\
.\\
\hat{y_{n}}
\end{bmatrix} = Xw = X(X^{T}X)^{-1}X^{T}y
\end{equation}$

### Ejemplo.
Tenemos la siguiente tabla:
| i  | x1(Rate) | x2(Temperature) | x3(Acid concentration) | y (Stack loss) |
|----|----------|-----------------|------------------------|----------------|
| 1  | 80       | 27              | 88                     | 37             |
| 2  | 62       | 22              | 87                     | 18             |
| 3  | 62       | 23              | 87                     | 18             |
| 4  | 62       | 24              | 93                     | 19             |
| 5  | 62       | 24              | 93                     | 20             |
| 6  | 58       | 23              | 87                     | 15             |
| 7  | 58       | 18              | 80                     | 14             |
| 8  | 58       | 18              | 89                     | 14             |
| 9  | 58       | 17              | 88                     | 13             |
| 10 | 58       | 18              | 82                     | 11             |
| 11 | 58       | 19              | 93                     | 12             |
| 12 | 50       | 18              | 89                     | 8              |
| 13 | 50       | 18              | 86                     | 7              |
| 14 | 50       | 19              | 72                     | 8              |
| 15 | 50       | 19              | 79                     | 8              |
| 16 | 50       | 20              | 80                     | 9              |
| 17 | 50       | 20              | 82                     | 15             |

Creamos la matriz $x$ y el vector $y$, llenando de 1's la primera columna de $x$:

$X = \begin{bmatrix}
1 & 80 & 27 & 88\\
1 & 62 & 27 & 87\\
. & . & . & .\\
. & . & . & .\\
. & . & . & .\\
1 & 50 & 20 & 80\\
1 & 56 & 20 & 82\\
\end{bmatrix}$

calculamos la operación por su traspuesta, dado que $X$ es dimensión $(m,n+1)$ y $X^{T}$ es dimensión $(n+1,m)$

$y = \begin{bmatrix}

\end{bmatrix}$

In [8]:
from typing import List
def ones(matrix: List) -> List:
    for fila in matrix:
        fila.insert(0, 1)

matriz = [[2,3,4],[5,6,7]]
print(matriz)
ones(matriz)
print(matriz)

[[2, 3, 4], [5, 6, 7]]
[[1, 2, 3, 4], [1, 5, 6, 7]]


In [9]:

from ipynb.fs.full.C9_Vectores_y_Matrices import dimension, mult_matrices, transpose

In [24]:
from ipynb.fs.full.C9_Vectores_y_Matrices import dot

In [11]:
X = [[80,27,88],[62, 22, 87],[62, 23, 87],
    [62, 24, 93],[62, 24, 93],[58, 23, 87],
    [58, 18, 80],[58, 18, 89],[58, 17, 88],
    [58, 18, 82],[58, 19, 93],[50, 18, 89],
    [50, 18, 86],[50, 19, 72],[50, 19, 79],
    [50, 20, 80],[60, 20, 82]
    ]

y = [37, 18, 18, 19, 20, 15, 14, 14, 
    13, 11, 12, 8, 7, 8, 8, 9, 15]
#print(ones(X), transpose(ones(X)))

llenamos a $X$ de 1's:

In [13]:
ones(X)
X

[[1, 80, 27, 88],
 [1, 62, 22, 87],
 [1, 62, 23, 87],
 [1, 62, 24, 93],
 [1, 62, 24, 93],
 [1, 58, 23, 87],
 [1, 58, 18, 80],
 [1, 58, 18, 89],
 [1, 58, 17, 88],
 [1, 58, 18, 82],
 [1, 58, 19, 93],
 [1, 50, 18, 89],
 [1, 50, 18, 86],
 [1, 50, 19, 72],
 [1, 50, 19, 79],
 [1, 50, 20, 80],
 [1, 60, 20, 82]]

Calculamos $X^{T}X$:

In [20]:
X_traspuesta = transpose(X)
Xt_X = mult_matrices(X_traspuesta, X)
print(Xt_X)

Matriz m2 traspuesta: [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [80, 62, 62, 62, 62, 58, 58, 58, 58, 58, 58, 50, 50, 50, 50, 50, 60], [27, 22, 23, 24, 24, 23, 18, 18, 17, 18, 19, 18, 18, 19, 19, 20, 20], [88, 87, 87, 93, 93, 87, 80, 89, 88, 82, 93, 89, 86, 72, 79, 80, 82]]
[[17, 986, 347, 1455], [986, 58060, 20380, 84682], [347, 20380, 7215, 29796], [1455, 84682, 29796, 125053]]


Le calculamos la inversa:

In [21]:
inversa = np.linalg.inv(np.array(Xt_X))
inversa

array([[ 1.42822245e+01, -6.13827645e-03, -2.89447806e-02,
        -1.55121400e-01],
       [-6.13827645e-03,  2.78637048e-03, -4.87789686e-03,
        -6.53182397e-04],
       [-2.89447806e-02, -4.87789686e-03,  1.73005133e-02,
        -4.82206554e-04],
       [-1.55121400e-01, -6.53182397e-04, -4.82206554e-04,
         2.37005313e-03]])

Ahora haremos $X^{T}y$:

In [23]:
X_traspuesta

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [80, 62, 62, 62, 62, 58, 58, 58, 58, 58, 58, 50, 50, 50, 50, 50, 60],
 [27, 22, 23, 24, 24, 23, 18, 18, 17, 18, 19, 18, 18, 19, 19, 20, 20],
 [88, 87, 87, 93, 93, 87, 80, 89, 88, 82, 93, 89, 86, 72, 79, 80, 82]]

In [25]:
def mult_matriz_vector(m1, v):
    filas = len(m1)
    salida = []
    for i in range(filas):
        salida[i].append(dot(m1[i], v))
    return salida

In [28]:
Xt_y = np.dot(X_traspuesta, y)
print(Xt_y)

[  246 15092  5295 21320]


finalmente hallemos el vector $w$, multiplicando Xt_y por la inversa:

In [29]:
np.dot(inversa, Xt_y)

array([-39.66250309,   0.78757477,   0.58793868,  -0.04144391])

Estos valores son los de $b_0$, $b_1$ y sucesivamente, de manera que la ecuación es:

$$\hat{y} = -39.662 + 0.787x_1 + 0.587x_2 - 0.041x_3$$

Probemos con la librería de Scikit-learn

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [50]:
reg = LinearRegression().fit(np.array(X), np.array(y))

Éste es el $R^{2}$

In [51]:
reg.score(X, y)

0.9787484853667733

In [52]:
reg.coef_

array([ 0.        ,  0.78757477,  0.58793868, -0.04144391])

In [53]:
reg.intercept_

-39.6625030916248

In [54]:
y_hat = reg.predict(X)
y_hat

array([35.57075852, 18.49616322, 19.0841019 , 19.42337711, 19.42337711,
       15.93380283, 13.2842168 , 12.91122161, 12.36472684, 13.20132898,
       13.33338464,  6.61062347,  6.7349552 ,  7.90310863,  7.61300125,
        8.15949602, 15.95235588])

También podemos calcular el $R^{2}$ así:

In [55]:
r2_score(y, y_hat)

0.9787484853667733

Y el error cuadrático medio

In [56]:
mean_squared_error(y, y_hat)

1.020366841005721